### Globals

In [17]:
import os
import glob
import subprocess
import sys

rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/'
qcdir = os.path.join(rootdir, 'qc', 'dMRI')

cluster_log_dir = '/home/preclineu/ramcir/DCCN/TorqueLogs'

cmd_qsub_base = ['/home/preclineu/ramcir/DCCN/Scripts/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '2gb',
                 '-logfiledir', cluster_log_dir
                ]

sub_dirs = sorted(glob.glob(os.path.join(rootdir,'subjects','*')))
print('Found', len(sub_dirs), 'subjects to process')


Found 20 subjects to process


### Sort subjects

In [27]:
sub_no_dmri = []
sub_ready = []
sub_complete = []

# Control the number of subjects
sdirs = sub_dirs[0:]

for s in sdirs:
    subid = os.path.splitext(os.path.basename(s))[0]

    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    ssepath = os.path.join(dmridir, 'dti_sse.nii.gz')  
    
    try:
        #Check if dMRI data is avalable
        if os.path.exists(dmridir) == False:  
#             print(subid,' dMRI data not found.')
            sub_no_dmri.append(subid)
        else:  
            #Check if quad has already been run
            if os.path.exists(ssepath) == True:
#                print(subid,' DTIfit already done.') 
                sub_complete.append(s)
            else:
#                print(subid,' is ready for DTIfit') 
                 sub_ready.append(s)
    except:
        print(subid, ' ignored due to', sys.exc_info()[0], '. Continuing with the next subject.')
        
print('Found', len(sub_no_dmri), 'subjects who are missing dMRI data')
print('Found', len(sub_ready), 'subjects ready to process')
print('Found', len(sub_complete), 'subjects already processed')

Found 1 subjects who are missing dMRI data
Found 18 subjects ready to process
Found 1 subjects already processed


### Run DTIfit

In [34]:
for s in sub_ready[0:]:
    subid = os.path.splitext(os.path.basename(s))
    
    #create paths for all files used in bash command
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    ssepath = os.path.join(dmridir, 'dti_sse.nii.gz ')
    dwi_b1000_new_vecs = os.path.join(dmridir, 'dwi_b1000.new_bvecs ')
    dwi_b1000_new_val = os.path.join(dmridir, 'dwi_b1000.new_val ')
    bvals = os.path.join(dmridir, 'bvals ')
    bvecs = os.path.join(dmridir, 'bvecs ')
    data = os.path.join(dmridir, 'data_ud.nii.gz ')
    dwi_b1000 = os.path.join(dmridir, 'dwi_b1000.nii.gz ')
    dwi_b1000_mask = os.path.join(dmridir, 'dwi_b1000_mask.nii.gz ')
    
    #check if sse already exists
    if os.path.exists(ssepath) == False:
        
        #comand for extracting b=1000 volumes
        dwiextract_cmd = ['dwiextract --export_grad_fsl ' + dwi_b1000_new_vecs + dwi_b1000_new_val + '-fslgrad',
                           bvecs + bvals + '-shell 0.0,1000.0 ' + data, dwi_b1000]
        
        #command for creating mask of b=1000 volumes
        bet_cmd = ['bet ' + dwi_b1000 + dwi_b1000_mask + '-f 0.50 -m -R']
    
        #command for DTIfit - creating dti parameters including sse
        dtifit_cmd = ['dtifit ',
                  '-k ' + dwi_b1000,
                  '-o ' + dmridir + '/dti',
                  '-m ' + dwi_b1000_mask, 
                  '-r ' + dwi_b1000_new_vecs, 
                  '-b ' + dwi_b1000_new_val,
                  '--sse']
        
        #Join comands and sumbit process
        proc_cmd = '"%s"' % str(' '.join(dwiextract_cmd) + ' && ' + ' '.join(bet_cmd) + ' && ' + ' '.join(dtifit_cmd))
        cmd_qsub = cmd_qsub_base + ['-name', 'UKB_dtifit_', subid ,'-command', proc_cmd]
#         print(cmd_qsub)
        subprocess.run(' '.join(cmd_qsub), shell=True)
    else: continue

TypeError: sequence item 9: expected str instance, tuple found